# Table of Contents
- [Installation](#Installation)
- [Setup](#Setup)
- [Run.py](#Run-py)
- [Argo UI](#Argo-UI)

# <a name="Installation"></a> Installation

## Prerequisite

Before using [dflow](#https://github.com/deepmodeling/dflow), we need to install the following two things:
- Docker (Official installation instruction: https://docs.docker.com/desktop/mac/install/)
- Minikube (Official installation instruction: https://minikube.sigs.k8s.io/docs/start/)

## Install pydflow

In [1]:
!pip install pydflow

Once installed, restart the jupyter notebook kernel to make the installation to take effect.

# <a name="Setup"></a> Setup

## Minikube

Dflow runs on kubernetes (k8s), so we need to start minikube

In [2]:
!minikube start --memory 2048 --cpus 2

😄  minikube v1.25.2 on Darwin 12.5
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🏃  Updating the running docker "minikube" container ...
🐳  Preparing Kubernetes v1.23.3 on Docker 20.10.12 ...
    ▪ kubelet.housekeeping-interval=5m
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Argo-workflows

Dflow is built on [argo-workflow](https://github.com/argoproj/argo-workflows), so we need to setup argo engine in k8s:

1. To get started quickly, we can use the quick start manifest which will install Argo Workflows as well as some commonly used components:

In [4]:
!kubectl create ns argo #create argo namespace
!kubectl apply -n argo -f https://raw.githubusercontent.com/dptech-corp/dflow/master/manifests/quick-start-postgres.yaml

customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtaskresults.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtasksets.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io unchanged
serviceaccount/argo unchanged
serviceaccount/argo-server unchanged
serviceaccount/github.com unchanged
role.rbac.authorization.k8s.io/agent unchanged
role.rbac.authorization.k8s.io/argo-role unchanged
role.rbac.authorization.k8s.io/argo-server-role unchanged
role.rbac.authorization.k8s.io/executor unchanged
role.rbac.authorization.k8s.io/pod-manager unchanged
role.rbac.authorization.k8s.io/submi

2. To monitor the setup progress, we can look at the pod status

In [5]:
!kubectl get pod -n argo

NAME                                   READY   STATUS               RESTARTS      AGE
argo-server-78f47df69f-92x7w           1/1     Running              0             15m
helloworld-df4rw-duplicate-572548458   0/2     Completed            0             54d
helloworld-df4rw-hello-352145594       0/2     Completed            0             54d
httpbin-588f9cb6f7-7z5r2               1/1     Running              0             15m
minio-59cf898d6f-wqs7f                 1/1     Running              0             15m
postgres-869f7fbd7f-8dpv4              1/1     Running              11 (3d ago)   54d
predict-2gd55-2646084598               0/2     Error                0             49d
predict-2qn4g-643214579                0/2     Completed            0             49d
predict-2r5gz-3894776341               0/2     Error                0             49d
predict-2w7d2-3250632301               0/2     Error                0             49d
predict-789g4-872904460                0/2     Error  

train-qhm2v-1759954948                 0/2     Completed            0             17d
train-qhm2v-1828213590                 0/2     Completed            0             17d
train-rsq6s-3204675437                 0/2     Completed            0             46d
train-sjh4h-4116987539                 0/2     Error                0             45d
train-vlx4q-1613118125                 0/2     Completed            0             45d
train-xs8bs-2248130112                 0/2     Completed            0             17d
train-xs8bs-3989070442                 0/2     Completed            0             17d
train-zxflm-1378528497                 0/2     Init:Error           0             17d
train-zxflm-603572723                  0/2     Completed            0             17d
workflow-controller-74646f5d98-cbcfh   1/1     Running              0             15m


NOTE: This process might take a little while, depending on the internet speed. Once the `STATUS` of all pods is `RUNNING`, you can proceed with the next step.

3. Open a port-forward so you can access the UI:

    Since we need to keep this UI running, we can run this command in the terminal:
    
```shell
kubectl -n argo port-forward deployment/argo-server 2746:2746
```

We can access the Argo UI: https://127.0.0.1:2746

Security warning will be shown but we can safely ignore it. This is because we haven't add ceritificate to this address. 

4. Open a port-forward so you can access the artifact repository
    
    Open another terminal and run this, because you want to keep artifact respository running. Note that you don't need to ingress the artifact repository if you are not downloading or uploading artifact.
    
```shell
kubectl -n argo port-forward deployment/minio 9000:9000
```

In the previous steps, we finished installing and seting up the necessary tools and environments for dflow to run. In this section, we will prepare a simple python script using dflow.

As explained in the [readme](https://github.com/dptech-corp/dflow#122--op-template), OP template is the fundamental component in dflow. For this particular workflow above, we need two OP templates:

Import modules

In [6]:
import json
from typing import List
from dflow import (
    Workflow,
    Step,
    argo_range,
    SlurmRemoteExecutor,
    upload_artifact,
    download_artifact,
    InputArtifact,
    OutputArtifact,
    ShellOPTemplate
)
from dflow.python import (
    PythonOPTemplate,
    OP,
    OPIO,
    OPIOSign,
    Artifact,
    Slices
)
import subprocess, os, shutil, glob
from pathlib import Path
from typing import List

Define the screen class. This step will download all the .cif files that satisfy the screening criteria in screening code.
This part needs inputs including 'api' for accessing materials project, 'screening_code' for different screening criteria, 'dirct' for storing the screening results, and 'atom_init' as a matrix for machine learning step.

In [7]:
class Screen(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'dirct': str,
            'api': str,
            'screen_code': Artifact(Path),
            'atom_init': Artifact(Path),
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'DownloadCIF_output' : Artifact(Path),
        })

    @OP.exec_sign_check
    def execute(self, op_in: OPIO) -> OPIO:
        cmd = f'python {op_in["screen_code"]} {op_in["api"]} {op_in["dirct"]}'
        subprocess.call(cmd, shell=True)
        cmd = f'cp {op_in["atom_init"]} {op_in["dirct"]}'
        subprocess.call(cmd, shell=True)
        return OPIO({
            "DownloadCIF_output": Path(f'{op_in["dirct"]}'), 
        })

Define the train class. This step will use one of the screening results to train the cgcnn model. 
This part needs inputs including 'train_code' and 'train_data_set' from previous screening step. It will output the 'trained_model' for prediction in next step.

In [8]:
class Train(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            #'train_parameter': list,
            'train_code': Artifact(Path),
            'train_data_set': Artifact(Path),
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'trained_model' : Artifact(Path),
        })

    @OP.exec_sign_check
    def execute(self, op_in: OPIO) -> OPIO:
        cmd = f'cd {op_in["train_code"]} && python main.py --train-size 50 --val-size 2 --test-size 1 {op_in["train_data_set"]}'
        subprocess.call(cmd, shell=True)
        return OPIO({
            "trained_model": Path(op_in["train_code"])/'model_best.pth.tar'
        })

Define the predict class. This step will use the trained model in previous Train step and screening results from the Screen step. It will output the predict the materials property for the screening results.

In [9]:
class Predict(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'predict_code': Artifact(Path),
            'predict_model': Artifact(Path),
            'predict_data_set': Artifact(Path),
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'predict_result' : Artifact(Path),
        })

    @OP.exec_sign_check
    def execute(self, op_in: OPIO) -> OPIO:
        #cmd = f'cd {op_in["predict_model"]} && ls && cp model_best.pth.tar {op_in["predict_code"]} && cd .. && ls'
        #subprocess.call(cmd, shell=True)
        cmd = f'cd {op_in["predict_code"]} && python predict.py {op_in["predict_model"]} {op_in["predict_data_set"]} && ls && cd .. && ls'
        subprocess.call(cmd, shell=True)

        return OPIO({
            "predict_result": Path(op_in["predict_code"])/'test_results.csv' 
        })

Upload the screening codes in screen_name_ls, 'atom_init.json' as a matrix for machine learning, training code in train folder, and prediction code in predict folder
Setup the screen step in a list (step_screen_ls) for parallel computing.
Your image and API key is needed

In [11]:
screen_name_ls = ["screen_1.py", "screen_2.py"]
atom_init = upload_artifact("atom_init.json")
train_main = upload_artifact("train")
predict_main = upload_artifact("predict")
step_screen_ls = []
screen_list = []
for i, criteria in enumerate(screen_name_ls):
    screen_list.append(upload_artifact(screen_name_ls[i]))
    step_screen =  Step(
        f"screen-{i}",
        PythonOPTemplate(Screen, image = '--your-image--',),
        artifacts = {"atom_init": atom_init, \
                    "screen_code": screen_list[i]},
        parameters = {"dirct": screen_name_ls[i][:-3], "api": '--your-api-key--'},
    )
    step_screen_ls.append(step_screen)

Setup the train step. Here it use the screening results in 'step_screen_ls[0]', which can be changed as needed
Your image is needed

In [12]:
train = Step(
    "train",
    PythonOPTemplate(Train, image = '--your-image--',),
    artifacts={"train_code": train_main, \
               "train_data_set": step_screen_ls[0].outputs.artifacts["DownloadCIF_output"],},
)

Stepup the prediction step. Here it use the trained model and screening results in 'step_screen_ls[1]', which can be changed as needed
Your image is needed

In [13]:
predict = Step(
    "predict",
    PythonOPTemplate(Predict, image = '--your-image--',),
    artifacts={"predict_code": predict_main, \
               "predict_data_set": step_screen_ls[1].outputs.artifacts["DownloadCIF_output"],\
               "predict_model": train.outputs.artifacts["trained_model"],},
)

Add the steps and submit the workflow

In [14]:
wf = Workflow("print-hello")
wf.add(step_screen_ls)
wf.add(train)
wf.add(predict)
wf.submit()

Workflow has been submitted (ID: print-hello-tmd4p)


{'metadata': {'name': 'print-hello-tmd4p', 'generateName': 'print-hello-', 'namespace': 'argo', 'uid': 'b011faf7-6a00-422b-9eff-9e575905c0a8', 'resourceVersion': '1100633', 'generation': 1, 'creationTimestamp': '2022-08-08T16:53:27Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2022-08-08T16:53:27Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'print-hello-steps', 'inputs': {}, 'outputs': {}, 'metadata': {}, 'steps': [[{'name': 'screen-0', 'template': 'screen-lv6gj', 'arguments': {'parameters': [{'name': 'dirct', 'value': 'screen_1', 'description': '{"type": "<class \'str\'>"}'}, {'name': 'api', 'value': 'v8mU74QhZSIisN26', 'description': '{"type": "<class \'str\'>"}'}], 'artifac

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)

We should see the following once loaded.